# CMOR 461 / INDE 561 — Homework 8

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wweissbluth/CMOR-461-LOGISTICS-AND-SUPPLY-CHAIN-MANAGEMENT/blob/main/hw8/hw8.ipynb)


## Problem 1: B&B Bookstore Inventory Optimization

**Scenario:**  
B&B, a local bookstore, must decide how many copies of a new book to order. They can only place one order before the selling season, which lasts one year. Afterward, unsold books are scrapped. 

### Given:

- **Retail price:** $20  
- **Wholesale price (from publisher):** $11  
- **Publisher's raw-material cost:** $3.75  
- **Bookstore shipping & handling cost:** $1.20  
- **Demand:** Normally distributed  
  - Mean: 1200  
  - Standard deviation: 340  
- **Unsold book scrap value:** $0.65  
- **Loss-of-goodwill cost for unmet demand:**  
  - Bookstore: $9.00  
  - Publisher: $4.00  


### Questions:

---

**a)** What is the bookstore’s optimal order quantity?

---

Per the derivation provided in the Canvas files.

We want

$$
\frac{r-w-c_b+g_b}{r-v+g_b}=F(Q^*_r)
$$

Meaning the CDF of the demand must be equal to the `Crtical Ratio`

In [335]:
r = 20 # retail price

w = 11 # wholesale price
c_b = 1.2 # Handling cost

c = 3.75 # production cost

v = .65 # salvage value

g_r = 9 # good will cost (retailer)

# Underage Cost
C_u_r = r - w - c_b + g_r
print(f"Underage Cost: {C_u_r:.2f}")

# Overage Cost
C_o_r = w + c_b - v
print(f"Overage Cost: {C_o_r:.2f}")

critical_ratio_r = (C_u_r) / (C_u_r + C_o_r)
print(f"Critical Ratio: {critical_ratio_r:.2f}")

from scipy.stats import norm

# Inputs
mean_demand = 1200
std_demand = 340

Q_r = norm.ppf(critical_ratio_r, loc=mean_demand, scale=std_demand)

print(f"Optimal book order quantity for retailer {Q_r:.2f} ({int(critical_ratio_r*100)}% of demand)")


Underage Cost: 16.80
Overage Cost: 11.55
Critical Ratio: 0.59
Optimal book order quantity for retailer 1279.63 (59% of demand)


---

**b)** What are the expected profits for the bookstore and publisher using the optimal order quantity from part (a)? What is the total expected profit?

---

`Retailer:`
$$
s(Q) = \mathbb{E}[\min(Q, D)] = \mu - \int_Q^\infty (x - Q) f(x) \, dx = \mu - n(Q)
$$


In [336]:
def q1b():
    from scipy.stats import norm
    from scipy.integrate import quad

    # Parameters
    mu = mean_demand
    sigma = std_demand
    Q = Q_r

    # PDF of normal distribution
    f = lambda x: norm.pdf(x, loc=mu, scale=sigma)

    # Tail integral for n(Q) = ∫_Q^∞ (x - Q) f(x) dx
    integrand = lambda x: (x - Q) * f(x)
    n_Q_r, _ = quad(integrand, Q, float('inf'))

    # Expected sales
    expected_sales = mu - n_Q_r

    # Output
    print(f"Expected lost sales n(Q) = {n_Q_r:.2f}")
    print(f"Expected sales s(Q) = {expected_sales:.2f}")
    return expected_sales, n_Q_r

s_Q_r, n_Q_r = q1b()

Expected lost sales n(Q) = 99.53
Expected sales s(Q) = 1100.47


$$I(Q) = \mathbb{E}[\max(0, Q - D)] = \mathbb{E}[Q - \min(Q, D)] = Q - s(Q)$$

In [337]:
I_Q_r = Q_r - s_Q_r
print(f"Given order quantity {Q_r:.2f}, expected inventory: {I_Q_r:.2f}")

Given order quantity 1279.63, expected inventory: 179.16


$$\pi_r(Q) = r \cdot s(Q) + v \cdot I(Q) - (w + c_B) \cdot Q - g_B \cdot n(Q)$$

In [338]:
pi_r_Q_r = r*s_Q_r + v*I_Q_r - (w+c_b)*Q_r - g_r*n_Q_r
print(f"Retail price (r): {r}")
print(f"Expected sales (s_Q_r): {s_Q_r:.2f}")
print(f"Salvage value (v): {v}")
print(f"Expected inventory (I_Q_r): {I_Q_r:.2f}")
print(f"Wholesale price (w): {w}")
print(f"Handling cost (c_b): {c_b}")
print(f"Order quantity (Q_r): {Q_r:.2f}")
print(f"Goodwill cost (g_r): {g_r}")
print(f"Expected lost sales (n_Q_r): {n_Q_r:.2f}")

print(f"\n\nGiven order quantity {Q_r:.2f}, expected retailer profit: {pi_r_Q_r:.2f}")

Retail price (r): 20
Expected sales (s_Q_r): 1100.47
Salvage value (v): 0.65
Expected inventory (I_Q_r): 179.16
Wholesale price (w): 11
Handling cost (c_b): 1.2
Order quantity (Q_r): 1279.63
Goodwill cost (g_r): 9
Expected lost sales (n_Q_r): 99.53


Given order quantity 1279.63, expected retailer profit: 5618.64


`Supplier:`

$$\pi_s(Q) = (w - c) \cdot Q-g_p\cdot n(Q)

$$


In [339]:
g_s = 4

pi_w_Q_r = (w-c)*Q_r-g_s*n_Q_r
print(f"Given order quantity {Q_r:.2f}, expected supplier profit: {pi_w_Q_r:.2f}")

Given order quantity 1279.63, expected supplier profit: 8879.24


`Whole Supply Chain:`

In [340]:
pi_0_Q_r = pi_r_Q_r + pi_w_Q_r
print(f"Total supply chain profit given order quantity {Q_r:.2f}: {pi_0_Q_r:.2f}")

Total supply chain profit given order quantity 1279.63: 14497.88


---

**c)** What order quantity maximizes total expected profit for the supply chain? What is the optimal total expected profit?

---

$$
C_u = r-(w + c_b + c)+g_s + g_r 
$$

In [341]:
# Underage Cost - what you lose if you don't have enough stock
c_u_o = r - (w + c_b + c) + g_s + g_r 
print(f"\nUnderage Cost for supplier: {c_u_o:.2f}")



Underage Cost for supplier: 17.05


$$c_o = -v + (w + c_b + c)$$

In [342]:
# Overage Cost - what you lose if you have too much stock
c_o_o = -v + (w + c_b + c)
print(f"Overage Cost for supplier: {c_o_o:.2f}")

Overage Cost for supplier: 15.30


Here we want the critical ratio to be the following:

$$
F_D(Q_r) = \alpha = \frac{C_u}{C_u + C_o}
$$


In [343]:
critical_ratio_0 = (c_u_o) / (c_u_o + c_o_o)
print(f"\nCritical Ratio for supplier: {critical_ratio_0:.2f}")


Critical Ratio for supplier: 0.53


$$Q_r = \mu + z_\alpha \cdot \sigma$$


In [344]:
Q_0 = norm.ppf(critical_ratio_0, loc=mean_demand, scale=std_demand)
print(f"\nOptimal book order quantity for supply chian {Q_r:.2f} ({int(critical_ratio_r*100)}% of demand)")


Optimal book order quantity for supply chian 1279.63 (59% of demand)


`Retailer:`

$$

s(Q) = \mathbb{E}[\min(Q, D)] = \mu - \int_Q^\infty (x - Q) f(x) \, dx = \mu - n(Q)

$$

In [345]:
def q1b():
    from scipy.stats import norm
    from scipy.integrate import quad

    # Parameters
    mu = mean_demand
    sigma = std_demand
    Q = Q_0

    # PDF of normal distribution
    f = lambda x: norm.pdf(x, loc=mu, scale=sigma)

    # Tail integral for n(Q) = ∫_Q^∞ (x - Q) f(x) dx
    integrand = lambda x: (x - Q) * f(x)
    n_Q_0, _ = quad(integrand, Q, float('inf'))

    # Expected sales
    expected_sales = mu - n_Q_0

    # Output
    print(f"Expected lost sales n(Q) = {n_Q_0:.2f}")
    print(f"Expected sales s(Q) = {expected_sales:.2f}")
    return expected_sales, n_Q_0

s_Q_0, n_Q_0 = q1b()

Expected lost sales n(Q) = 124.42
Expected sales s(Q) = 1075.58


$$I(Q) = \mathbb{E}[\max(0, Q - D)] = \mathbb{E}[Q - \min(Q, D)] = Q - s(Q)$$

In [346]:
I_Q_0 = Q_0 - s_Q_0
print(f"Given order quantity {Q_0:.2f}, expected inventory: {I_Q_0:.2f}")

Given order quantity 1223.07, expected inventory: 147.49


$$\pi_r(Q) = r \cdot s(Q) + v \cdot I(Q) - (w + c_B) \cdot Q - g_B \cdot n(Q)$$

In [347]:
pi_r_Q_0 = r*s_Q_0 + v*I_Q_0 - (w+c_b)*Q_0 - g_r*n_Q_0
print(f"Retail price (r): {r}")
print(f"Expected sales (s_Q_r): {s_Q_r:.2f}")
print(f"Salvage value (v): {v}")
print(f"Expected inventory (I_Q_r): {I_Q_r:.2f}")
print(f"Wholesale price (w): {w}")
print(f"Handling cost (c_b): {c_b}")
print(f"Order quantity (Q_r): {Q_r:.2f}")
print(f"Goodwill cost (g_r): {g_r}")
print(f"Expected lost sales (n_Q_r): {n_Q_r:.2f}")

print(f"\n\nGiven order quantity {Q_r:.2f}, expected retailer profit: {pi_r_Q_r:.2f}")

Retail price (r): 20
Expected sales (s_Q_r): 1100.47
Salvage value (v): 0.65
Expected inventory (I_Q_r): 179.16
Wholesale price (w): 11
Handling cost (c_b): 1.2
Order quantity (Q_r): 1279.63
Goodwill cost (g_r): 9
Expected lost sales (n_Q_r): 99.53


Given order quantity 1279.63, expected retailer profit: 5618.64


`Supplier:`

$$\pi_s(Q) = (w - c) \cdot Q-g_p\cdot n(Q)

$$


In [348]:
pi_w_Q_0 = (w-c)*Q_0-g_s*n_Q_0
print(f"Given order quantity {Q_0:.2f}, expected supplier profit: {pi_w_Q_0:.2f}")

Given order quantity 1223.07, expected supplier profit: 8369.58


`Whole Supply Chain:`

In [349]:
pi_0_Q_0 = pi_r_Q_0 + pi_w_Q_0
print(f"Total supply chain profit given order quantity {Q_0:.2f}: {pi_0_Q_0:.2f}")

Total supply chain profit given order quantity 1223.07: 13935.89


---

In [350]:
s_Q_0=min(mean_demand, Q_0)
print(f"\tExpected sales after order quantity Q_0: {s_Q_0:.2f}")

I_Q_0 = max(0, Q_0 - mean_demand)
print(f"\tExpected Inventory remaining after sales: {I_Q_0:.2f}")

pi_r_Q_0 = r*s_Q_0 + v*I_Q_0 - (w+c_b)*Q_0
print(f"\n\tRetailer Profit from stock level Q_0: ${pi_r_Q_0:.2f} USD")

pi_w_Q_0 = (w-c)*Q_r
print(f"\tWholesaler Profit from stock level Q_0: ${pi_w_Q_0:.2f} USD")

pi_0_Q_0 = pi_r_Q_0+pi_w_Q_0
print(f"Total Supply chain profit: ${pi_0_Q_r:.2f} USD")

	Expected sales after order quantity Q_0: 1200.00
	Expected Inventory remaining after sales: 23.07

	Retailer Profit from stock level Q_0: $9093.55 USD
	Wholesaler Profit from stock level Q_0: $9277.35 USD
Total Supply chain profit: $14497.88 USD


---

**Buyback Contract Scenario:**  
The publisher offers a buyback credit of **$8**.

**d)** What is the bookstore’s optimal order quantity under this buyback contract? What are the expected profits for the bookstore and the publisher?

---

---

**e)** What buyback price will coordinate the supply chain? What are the resulting profits?

---

---

**Revenue Sharing Contract Scenario:**  
The bookstore keeps **60%** of its revenue, and gives **40%** to the publisher.

**f)** What wholesale price will coordinate the supply chain, and what will be the resulting profits?

---

## Problem 2: Two-Stage Ordering in a Supply Chain

**Scenario:**  
A supply chain has a single supplier and single retailer. The retailer can place **two orders**:  
1. Before demand is realized.  
2. After demand is realized (only if the first order is insufficient).  
Any unmet demand before the second order incurs a loss-of-goodwill cost `p`.

### Given:

- Demand is random with:
  - PDF: `f`
  - CDF: `F`
  - Mean: `μ`
- Selling price per item: `r`  
- Wholesale price per item (both orders): `w`  
- Salvage value per unit: `v`  
- First order production cost: `c₁`  
- Second order production cost: `c₂`  
- Assumptions:  
  - `c₁ < c₂`  
  - `v < c₁`  
  - `c₂ < w`  
- The manufacturer **produces to order** (no inventory).

### Sequence of Events:

1. Retailer places **first order** of size `Q`.
2. Order is **delivered immediately**.
3. **Demand is realized**.  
   - Demand ≤ Q: sales fulfilled immediately.  
   - Demand > Q: unmet demand is backordered.
4. Retailer places **second order** for unmet demand.
5. Order is **delivered immediately**, all unmet demand fulfilled.
6. **Unsold inventory** is salvaged.

### Questions:

---

**a)** Write the expected profit functions:
- Retailer’s profit: `π_r(Q)`  
- Supplier’s profit: `π_s(Q)`  
- Total supply chain profit: `Π(Q)`

---

---

**b)** Prove that the retailer’s optimal order quantity is strictly smaller than the supply chain’s optimal order quantity.  
*Conclusion: the supply chain is not coordinated.*

---

---

**c)** Consider a **buyback contract**:  
- Retailer pays wholesale price `w`  
- Supplier reimburses the retailer `b` per unsold unit  
- Let:  
  ```math
  w(b) = b + v + (p / (c₁ - v)) * (p + c₂ - c₁)
  ```
  Prove this contract **coordinates the supply chain**.  
  *Verify necessary and sufficient conditions.*

---

---

**d)** Briefly explain:
- Why the original supply chain was **not coordinated**.
- Why the **buyback contract** coordinates it.

---